## Optimal Weights Calculation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
returns = pd.read_excel('MSGI-Asset-Class-Optimization-Merged-Real-Estate-20190628_V2.xlsx',sheet_name='Original DataSet',\
                        usecols='V:AC')
returns = returns.dropna()
returns = returns.iloc[1:]
returns = returns.rename(columns={"R12MO": "BDC", "R12MO.1": 'Equity',"R12MO.2":'Hedge Fund','R12MO.3':'High Yield Corporates',\
                                                   'R12MO.4':'MLP','R12MO.5':'Preferred Stocks','R12MO.6':'Real Estate','R12MO.7':'REIT'})
returns = returns.reset_index(drop=True)
returns.head() # use 12-month rolling-window returns (annual return) 
# Save to all of the four xlsx files

,BDC,Equity,Hedge Fund,High Yield Corporates,MLP,Preferred Stocks,Real Estate,REIT
0,-0.432904,-0.276086,-0.154262,-0.263888,-0.369146,-0.257633,-0.43406,-0.377276
1,-0.527048,-0.365222,-0.125196,-0.21414,-0.268635,-0.406092,-0.577302,-0.47972
2,-0.650579,-0.414943,-0.149698,-0.232256,-0.295277,-0.535508,-0.588474,-0.572867
3,-0.591145,-0.362867,-0.122461,-0.202762,-0.242947,-0.420901,-0.554229,-0.58159
4,-0.48151,-0.309194,-0.123326,-0.146906,-0.217037,-0.358516,-0.431758,-0.482144


In [2]:
mean_returns_out = returns.mean()
mean_returns_out = pd.DataFrame(mean_returns_out,columns=['Expected Return'])
# take the mean return of each asset class as its expected return
# Save to all of the four xlsx files
returns = returns.astype(float)
cov_matrix = returns.cov() 
# calculate the covariance matrix according to historical 12-month rolling window returns
# Save to all of the four xlsx files
cov_matrix_out = cov_matrix.astype(float)
mean_returns = mean_returns_out.to_numpy().reshape(1,8)
cov_matrix = cov_matrix_out.to_numpy()
returns_out = returns.head(n=10)

### 1. Optimal Weights with No Constraints

In [3]:
# No Constraints
# Set the number of iterations to 10000 and define an array to hold the simulation results; initially set to all zeros
num_iterations = 10000
simulation_res = np.zeros((4+returns.shape[1]-1,num_iterations))
for i in range(num_iterations):
# Select random weights and normalize to set the sum to 1
    weights = np.array(np.random.random(8))
    weights /= np.sum(weights)
# Calculate the return and standard deviation for every step
    portfolio_return = np.sum(mean_returns * weights)
    portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))
# Store all the results in a defined array
    simulation_res[0,i] = portfolio_return
    simulation_res[1,i] = portfolio_std_dev
# Calculate Sharpe ratio and store it in the array
    simulation_res[2,i] = simulation_res[0,i] / simulation_res[1,i]
# Save the weights in the array
    for j in range(len(weights)):
            simulation_res[j+3,i] = weights[j]

sim_frame = pd.DataFrame(simulation_res.T,columns=['ret','stdev','sharpe','BDC','Equity','Hedge Fund','High Yield Corporates',\
                                                   'MLP','Preferred Stocks','Real Estate','REIT'])

In [4]:
# Spot the position of the portfolio with highest Sharpe Ratio
max_sharpe = sim_frame.iloc[sim_frame['sharpe'].idxmax()]
max_sharpe =  pd.DataFrame(np.array(max_sharpe).reshape(1,11),columns=['ret','stdev','sharpe','BDC','Equity','Hedge Fund',\
                                                                       'High Yield Corporates','MLP','Preferred Stocks','Real Estate','REIT'],index=["Value/Weights"])
# Sharpe-Ratio Maximizing Portfolio with No Constraints on Weights
# Save to "Optimal Weights_No Constraint.xlsx"

In [5]:
writer = pd.ExcelWriter('Optimal Weights_No Constraints.xlsx')
returns_out.to_excel(writer, '12-month returns_Head')
mean_returns_out.to_excel(writer, 'Expected Return')
cov_matrix_out.to_excel(writer, 'Covariance Matrix')
max_sharpe.to_excel(writer,'Sharpe Maximizing Portfolio')
writer.save()

### 2. Optimal Weights with Constraints

#### a.	Minimize the Portfolio Risk for the Desired Level of Expected Portfolio Return

In [6]:
# With Constraints:x1>=5%, x2<=20%, x3<=15%,x8>=25%, No shorts allowed
# Define Matrix
a = np.ones((1,8)) # constraint 1: sum weight = 1
b = -1*np.eye(8)  # constriant 2: no short allowed
c = -1*mean_returns # constraint 3: portfolio's expected return > last period's portfolio return
d = np.array([[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1]]) 
# constraint 4: constraints on weight of x1,x2,x3,x8

In [7]:
from cvxopt import matrix, solvers
# Use Quadratic Programming Method
P = 2*matrix(cov_matrix)
# Matrix for Objective Function, Quadratic Part
q = matrix(np.zeros((8,1))) 
# Matrix for Objective Function, Linear Part
A = matrix(a) 
# Matrix for the Equality Constraint, Coefficient Part (Left Hand)
B = matrix([1.0]) 
# Matrix for the Equality Constraint, Value (Right Hand)
G = matrix(np.concatenate((b,c,d), axis=0)) 
# Matrix for the Inequality Constraint, Coefficient Part (Left Hand)
h = matrix([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-0.104,0.2,0.15,-0.05,-0.25]) 
# Matrix for the Inequality Constraint, Value (Right Hand)

In [8]:
sol = solvers.qp(P,q,G,h,A,B)   #  use function solvers.qp to solve the problem
weights = np.array(sol['x'])
# Optimal Weights solution for this problem
Rp = np.dot(mean_returns, weights) 
# Protfolio Return with optimal weights
Std_p = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) 
# Protfolio standard deviation with optimal weights
rf = 0.021
# risk-free rate as of 2019
Sharpe = (Rp-rf)/Std_p
# Protfolio Sharpe Ratio with optimal weights
p_result = np.vstack((Rp,Std_p,Sharpe,weights))
result_1 = pd.DataFrame(p_result.T,columns=['Return','Stdev','Sharpe','BDC','Equity','Hedge Fund','High Yield Corporates',\
                                            'MLP','Preferred Stocks','Real Estate','REIT'],index=["Value/Weights"])
# Sharpe-Ratio Maximizing Portfolio with Constraints on Weights
# Save to "Optimal Weights_With Constraints_Method_1.xlxs"

     pcost       dcost       gap    pres   dres
 0:  2.6820e-02 -9.6331e-01  2e+01  4e+00  4e+00
 1:  7.0273e-02 -1.0621e+00  2e+00  2e-01  3e-01
 2:  5.4475e-02 -1.2869e-01  2e-01  2e-16  3e-15
 3:  4.8422e-02  7.5609e-03  4e-02  2e-16  6e-16
 4:  3.0236e-02  3.9027e-03  3e-02  2e-16  2e-16
 5:  2.6363e-02  2.1660e-02  5e-03  6e-17  2e-15
 6:  2.4942e-02  2.4230e-02  7e-04  1e-16  4e-16
 7:  2.4768e-02  2.4746e-02  2e-05  1e-16  2e-15
 8:  2.4763e-02  2.4762e-02  5e-07  2e-16  2e-14
 9:  2.4763e-02  2.4763e-02  5e-09  7e-17  8e-14
Optimal solution found.


In [9]:
writer = pd.ExcelWriter('Optimal Weights_With Constraints_Method_1.xlsx')
returns_out.to_excel(writer, '12-month returns_Head')
mean_returns_out.to_excel(writer, 'Expected Return')
cov_matrix_out.to_excel(writer, 'Covariance Matrix')
result_1.to_excel(writer,'Sharpe Maximizing Portfolio')
writer.save()

#### b. Maximize Utility Function

In [10]:
# With Constraints:x1>=5%, x2<=20%, x3<=15%,x8>=25%, No shorts allowed
# Define Matrix
a = np.ones((1,8)) # constraint 1: sum weight = 1
b = -1*np.eye(8)  # constriant 2: no short allowed
c = mean_returns.T # asset class's expected return vector
d = np.array([[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1]]) 
# constraint 3: constraints on weight of x1,x2,x3,x8

In [11]:
from cvxopt import matrix, solvers
lamda = 3 # risk-aversion level of investors
P = lamda*matrix(cov_matrix) 
# Matrix for Objective Function, Quadratic Part
q = -1*matrix(c) 
# Matrix for Objective Function, Linear Part
A = matrix(a) 
# Matrix for the Equality Constraint, Coefficient Part
B = matrix([1.0]) 
# Matrix for the Equality Constraint, Value
G = matrix(np.concatenate((b,d), axis=0)) 
# Matrix for the Inequality Constraint, Coefficient Part
h = matrix([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.2,0.15,-0.05,-0.25]) 
# Matrix for the Inequality Constraint, Value

In [12]:
sol = solvers.qp(P,q,G,h,A,B)   # use function solvers.qp to solve
weights = np.array(sol['x']) 
# Optimal Weights solution for this problem
Rp = np.dot(mean_returns, weights) 
# Protfolio Return with optimal weights
Std_p = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) 
# Protfolio standard deviation with optimal weights
rf = 0.021 # risk-free rate as of 2019
Sharpe = (Rp-rf)/Std_p 
# Protfolio Sharpe Ratio with optimal weights
p_result = np.vstack((Rp,Std_p,Sharpe,weights)) 
result_2 = pd.DataFrame(p_result.T,columns=['Return','Stdev','Sharpe','BDC','Equity','Hedge Fund','High Yield Corporates',\
                                            'MLP','Preferred Stocks','Real Estate','REIT'],index=["Value/Weights"])
result_2
# Sharpe-Ratio Maximizing Portfolio with Constraints on Weights
# Save to "Optimal Weights_With Constraints_Method_2.xlsx"

     pcost       dcost       gap    pres   dres
 0: -6.0369e-02 -1.0672e+00  2e+01  4e+00  4e+00
 1: -1.6712e-02 -1.1342e+00  2e+00  2e-01  2e-01
 2: -3.2057e-02 -1.6753e-01  1e-01  2e-16  2e-15
 3: -4.2134e-02 -8.4057e-02  4e-02  1e-16  5e-16
 4: -5.2862e-02 -9.1272e-02  4e-02  2e-16  4e-16
 5: -6.6076e-02 -8.0715e-02  1e-02  1e-16  1e-16
 6: -6.7069e-02 -6.8001e-02  9e-04  1e-16  2e-17
 7: -6.7467e-02 -6.7482e-02  2e-05  8e-17  2e-17
 8: -6.7474e-02 -6.7474e-02  2e-07  1e-16  3e-17
 9: -6.7474e-02 -6.7474e-02  2e-09  1e-16  2e-17
Optimal solution found.


,Return,Stdev,Sharpe,BDC,Equity,Hedge Fund,High Yield Corporates,MLP,Preferred Stocks,Real Estate,REIT
Value/Weights,0.10784,0.164046,0.529366,0.05,0.2,5.523919e-08,0.457013,1.720071e-09,2.405363e-09,0.042987,0.25


In [13]:
writer = pd.ExcelWriter('Optimal Weights_With Constraints_Method_2.xlsx')
returns_out.to_excel(writer, '12-month returns_Head')
mean_returns_out.to_excel(writer, 'Expected Return')
cov_matrix_out.to_excel(writer, 'Covariance Matrix')
result_2.to_excel(writer,'Sharpe Maximizing Portfolio')
writer.save()

#### c.	Maximize Sharpe Ratio Using Iterative Method for Expected Return

In [14]:
num = 2000 # number of iteration
simulation = np.zeros((num,4+returns.shape[1]-1)) # results should be 2000*15 matrix
ret_range = np.linspace(0.08,0.2, num) # portfolio expected return ranges from 8% to 20%

In [15]:
# With Constraints:x1>=5%, x2<=20%, x3<=15%,x8>=25%, No shorts allowed
# Define Matrix
a = np.ones((1,8)) 
# constraint 1: sum weight = 1
b = -1*np.eye(8)  
# constriant 2: no short allowed
c = mean_returns 
# constraint 3: portfolio's expected return > each value in ret_range
d = np.array([[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[-1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,-1]])
# constraint 4: constraints on weight of x1,x2,x3,x8

In [16]:
from cvxopt import matrix, solvers
P = 2*matrix(cov_matrix) 
# Matrix for Objective Function, Quadratic Part
q = matrix(np.zeros((8,1))) 
# Matrix for Objective Function, Linear Part
A = matrix(np.concatenate((a,c))) 
# Matrix for the Equality Constraint, Coefficient Part
G = matrix(np.concatenate((b,d), axis=0)) 
# Matrix for the Inequality Constraint, Coefficient Part
h = matrix([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.15,-0.05,-0.25])
# Matrix for the Inequality Constraint, Value

In [17]:
for i in range(2000):
    B = matrix([1.0,ret_range[i]]) 
    # Matrix for the Equality Constraint, Value
    sol = solvers.qp(P,q,G,h,A,B)   
    # use function solvers.qp to solve
    weights = np.array(sol['x']) 
    # Optimal Weights solution for this problem
    simulation[i,0] = np.dot(mean_returns, weights) 
    # Protfolio Return with optimal weights
    simulation[i,1] = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) 
    # Protfolio standard deviation with optimal weights
    rf = 0.021
    simulation[i,2] = (simulation[i,0]-rf)/simulation[i,1]
    # Protfolio Sharpe Ratio with optimal weights
    for j in range(len(weights)):
        simulation[i,j+3] = weights[j]

     pcost       dcost       gap    pres   dres
 0:  1.6564e-02 -6.2878e-01  2e+01  4e+00  5e+00
 1:  2.8739e-02 -5.2205e-01  3e+00  6e-01  6e-01
 2:  2.6101e-02  8.5922e-01  2e+00  3e-01  4e-01
 3:  3.3948e-02  4.6657e+00  2e+00  3e-01  3e-01
 4:  3.2972e-02  3.0760e+01  2e+00  2e-01  3e-01
 5:  3.2863e-02  6.1420e+02  9e+00  2e-01  3e-01
 6:  3.2756e-02  4.2561e+04  7e+01  2e-01  3e-01
 7:  3.2740e-02  2.6783e+07  1e+03  2e-01  3e-01
 8:  3.2740e-02  7.1629e+08  3e+02  2e-01  3e-01
 9:  4.1774e-02  6.8700e+13  1e+13  2e-01  2e-01
10:  3.5143e-02  9.5405e+13  2e+12  2e-02  3e+01
Terminated (singular KKT matrix).
     pcost       dcost       gap    pres   dres
 0:  1.6586e-02 -6.3034e-01  2e+01  4e+00  5e+00
 1:  2.8782e-02 -5.2464e-01  3e+00  6e-01  6e-01
 2:  2.6115e-02  8.4893e-01  2e+00  3e-01  4e-01
 3:  3.3941e-02  4.6280e+00  2e+00  3e-01  3e-01
 4:  3.2967e-02  3.0512e+01  2e+00  2e-01  3e-01
 5:  3.2859e-02  6.1015e+02  9e+00  2e-01  3e-01
 6:  3.2753e-02  4.2324e+04  7e+01  2

In [19]:
sim_frame = pd.DataFrame(simulation,columns=['ret','stdev','sharpe','BDC','Equity','Hedge Fund','High Yield Corporates',\
                                             'MLP','Preferred Stocks','Real Estate','REIT'])
#Spot the position of the portfolio with highest Sharpe Ratio
max_sharpe = sim_frame.iloc[sim_frame['sharpe'].idxmax()]
result_3 =  pd.DataFrame(np.array(max_sharpe).reshape(1,11),columns=['ret','stdev','sharpe','BDC','Equity','Hedge Fund',\
                                                                       'High Yield Corporates','MLP','Preferred Stocks',\
                                                                       'Real Estate','REIT'],index=["Value/Weights"])
result_3
# Sharpe-Ratio Maximizing Portfolio with Constraints on Weights
# Save to "Optimal Weights_With Constraints_Method_3.xlsx"

,ret,stdev,sharpe,BDC,Equity,Hedge Fund,High Yield Corporates,MLP,Preferred Stocks,Real Estate,REIT
Value/Weights,0.107614,0.163601,0.529422,0.05,0.2,0.000013,0.465508,1.102665e-08,1.917636e-08,0.034479,0.25


In [20]:
writer = pd.ExcelWriter('Optimal Weights_With Constraints_Method_3.xlsx')
returns_out.to_excel(writer, '12-month returns_Head')
mean_returns_out.to_excel(writer, 'Expected Return')
cov_matrix_out.to_excel(writer, 'Covariance Matrix')
result_3.to_excel(writer,'Sharpe Maximizing Portfolio')
writer.save()